<a href="https://colab.research.google.com/github/evolu-tion/Comparative-genomics/blob/main/Comparative_genomics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Guide to comparative genome analysis

In comparative genomics approach separates into 3 steps 
1. Define interested genes of pathway in template organism and preparation
 1. Retriving interseted gene using [KEGG API](https://www.kegg.jp/kegg/rest/keggapi.html)
 2. protein ID of interested genes using KEGG API
 3. Get amino acid sequences of template plant
2. Find orthologus genes via reciprocal best hit BLASTp
3. Functional assignment
4. Pathway visualization

# 0. Installation packages

In [ ]:
!pip install biopython
!pip install pandas
!mkdir -p input

In [ ]:
!mkdir -p required_package
!wget --quiet https://ftp.ncbi.nlm.nih.gov/blast/executables/blast+/2.11.0/ncbi-blast-2.11.0+-x64-linux.tar.gz -O required_package/ncbi-blast-2.11.0+-x64-linux.tar.gz
!tar xzvf required_package/ncbi-blast-2.11.0+-x64-linux.tar.gz --directory required_package
!mv required_package/ncbi-blast-2.11.0+/bin .

ncbi-blast-2.11.0+/
ncbi-blast-2.11.0+/ChangeLog
ncbi-blast-2.11.0+/LICENSE
ncbi-blast-2.11.0+/README
ncbi-blast-2.11.0+/bin/
ncbi-blast-2.11.0+/bin/blast_formatter
ncbi-blast-2.11.0+/bin/blastdb_aliastool
ncbi-blast-2.11.0+/bin/blastdbcheck
ncbi-blast-2.11.0+/bin/blastdbcmd
ncbi-blast-2.11.0+/bin/blastn
ncbi-blast-2.11.0+/bin/blastp
ncbi-blast-2.11.0+/bin/blastx
ncbi-blast-2.11.0+/bin/cleanup-blastdb-volumes.py
ncbi-blast-2.11.0+/bin/convert2blastmask
ncbi-blast-2.11.0+/bin/deltablast
ncbi-blast-2.11.0+/bin/dustmasker
ncbi-blast-2.11.0+/bin/get_species_taxids.sh
ncbi-blast-2.11.0+/bin/legacy_blast.pl
ncbi-blast-2.11.0+/bin/makeblastdb
ncbi-blast-2.11.0+/bin/makembindex
ncbi-blast-2.11.0+/bin/makeprofiledb
ncbi-blast-2.11.0+/bin/psiblast
ncbi-blast-2.11.0+/bin/rpsblast
ncbi-blast-2.11.0+/bin/rpstblastn
ncbi-blast-2.11.0+/bin/segmasker
ncbi-blast-2.11.0+/bin/tblastn
ncbi-blast-2.11.0+/bin/tblastx
ncbi-blast-2.11.0+/bin/update_blastdb.pl
ncbi-blast-2.11.0+/bin/windowmasker
ncbi-blast-2.1

In [ ]:
# Import Biopython modules to interact with KEGG
import pandas as pd
from Bio.KEGG import REST
from Bio import SeqIO

# 1. Define interested genes of pathway in template organism and preparation
##  A. Retriving interseted gene using KEGG API

In [ ]:
# defind organism id and pathway of your interested from https://www.kegg.jp/
kegg_organism_id = "ath"
kegg_pathway_interested = "ath00500"

# retreving list of gene in pathway of interested
url = "http://rest.kegg.jp/link/" + kegg_organism_id + "/" + kegg_pathway_interested
gene_in_pathway = pd.read_table(url, names=["pathway", "geneID"], header=None)
gene_in_pathway_list = list(gene_in_pathway["geneID"])

# retreving EC information of all genes
url = "http://rest.kegg.jp/link/ec/" + kegg_organism_id
gene_with_ec = pd.read_table(url, names=["geneID", "ec"], header=None, )

# merge EC to each gene
gene_in_pathway_list = pd.merge(gene_in_pathway, gene_with_ec, how='left', on='geneID')
gene_in_pathway_list

,pathway,geneID,ec
0,path:ath00500,ath:AT1G02800,ec:3.2.1.4
1,path:ath00500,ath:AT1G02850,ec:3.2.1.21
2,path:ath00500,ath:AT1G03310,ec:3.2.1.68
3,path:ath00500,ath:AT1G04920,ec:2.4.1.14
4,path:ath00500,ath:AT1G05610,ec:2.7.7.27
...,...,...,...
178,path:ath00500,ath:AT5G51830,ec:2.7.1.4
179,path:ath00500,ath:AT5G54570,ec:3.2.1.21
180,path:ath00500,ath:AT5G58090,ec:3.2.1.39
181,path:ath00500,ath:AT5G64860,ec:2.4.1.25


In [ ]:
# Download genome feature file annotation from NCBI datbase
!wget -q ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/735/GCF_000001735.3_TAIR10/GCF_000001735.3_TAIR10_feature_table.txt.gz \
     -O "input/GCF_000001735.3_TAIR10_feature_table.txt.gz"
# Download protein sequence in FASTA format from NCBI datbase
!wget -q ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/735/GCF_000001735.3_TAIR10/GCF_000001735.3_TAIR10_protein.faa.gz \
     -O "input/GCF_000001735.3_TAIR10_protein.faa.gz"

# Uncompresed both file
!gunzip input/GCF_000001735.3_TAIR10_protein.faa.gz
!gunzip input/GCF_000001735.3_TAIR10_feature_table.txt.gz

In [ ]:
template_gene_tbl = pd.read_table("input/GCF_000001735.3_TAIR10_feature_table.txt", low_memory=False) \
    .rename(columns={"# feature" : "feature"}) \
    .query("feature == 'CDS'") \
    .filter(["locus_tag", "product_accession"]) \
    .dropna() \
    .rename(columns={"locus_tag" : "geneID"})
template_gene_tbl

,geneID,product_accession
2,AT1G01010,NP_171609.1
10,AT1G01020,NP_001318899.1
11,AT1G01020,NP_001321777.1
12,AT1G01020,NP_001321775.1
13,AT1G01020,NP_001321776.1
...,...,...
146455,ArthCp088,NP_051118.1
146457,ArthCp086,NP_051119.2
146461,ArthCp083,NP_051121.1
146465,ArthCp084,NP_051122.1


In [ ]:
gene_in_pathway_list["geneID"] = gene_in_pathway_list["geneID"].replace('ath:', '', regex=True)
template_protein = pd.merge(gene_in_pathway_list, template_gene_tbl, how='left', on='geneID')
template_protein

,pathway,geneID,ec,product_accession
0,path:ath00500,AT1G02800,ec:3.2.1.4,NP_171779.1
1,path:ath00500,AT1G02850,ec:3.2.1.21,NP_973746.3
2,path:ath00500,AT1G02850,ec:3.2.1.21,NP_001117217.1
3,path:ath00500,AT1G02850,ec:3.2.1.21,NP_849578.5
4,path:ath00500,AT1G02850,ec:3.2.1.21,NP_563666.1
...,...,...,...,...
384,path:ath00500,AT5G58090,ec:3.2.1.39,NP_200617.2
385,path:ath00500,AT5G64860,ec:2.4.1.25,NP_201291.1
386,path:ath00500,AT5G65140,ec:3.1.3.12,NP_201319.2
387,path:ath00500,AT5G65140,ec:3.1.3.12,NP_001330820.1


In [ ]:
# display number of unique protein id
print("Number of unique protein id is", len(template_protein["product_accession"].unique()), "proteins")

Number of unique protein id is 361 proteins


In [ ]:
filename = "input/GCF_000001735.3_TAIR10_protein.faa"
fasta_record = SeqIO.to_dict(SeqIO.parse(filename, "fasta"))

select_fasta = ''
for gene in template_protein["product_accession"].unique():
    select_fasta = select_fasta + '>' + fasta_record[gene].name + '\n' + fasta_record[gene].seq + '\n'

f = open("input/seq_ara_STA.fasta", "w")
f.write(str(select_fasta))
f.close()

# 2. Find orthologus genes via reciprocal best hit BLASTp!

In [ ]:
# Make cassava protein database for BLASTp
!wget --quiet "https://drive.google.com/uc?export=download&id=1Lo07b7-naWuJvhnYR-AJD8_pxhridghf" -O input/Mesculenta_305_v6.1.protein.fa.gz
!gunzip input/Mesculenta_305_v6.1.protein.fa.gz

In [ ]:
!grep -c ^processor /proc/cpuinfo

8


In [ ]:
!bin/makeblastdb -in input/Mesculenta_305_v6.1.protein.fa \
            -dbtype prot \
            -title cassavaDB \
            -out input/cassavaDB
!bin/blastp -db input/cassavaDB \
        -query input/seq_ara_STA.fasta \
        -evalue 1e-10 \
        -max_target_seqs 1 \
        -num_threads 20  \
        -outfmt "6 std qcovhsp" \
        -out ara_cas.txt



Building a new DB, current time: 12/03/2020 16:34:53
New DB name:   /home/golf/input/cassavaDB
New DB title:  cassavaDB
Sequence type: Protein
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 41381 sequences in 0.748258 seconds.




In [ ]:
first_BLASTp = pd.read_table("ara_cas.txt", 
                             names=["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore", "qcovhsp"], 
                             header=None)
first_BLASTp

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,qcovhsp
0,NP_171779.1,Manes.01G221800.1.p,78.947,475,96,2,31,501,28,502,0.000000e+00,775,94
1,NP_973746.3,Manes.05G062800.1.p,61.010,495,157,5,22,511,20,483,0.000000e+00,629,94
2,NP_001117217.1,Manes.05G062800.1.p,61.134,494,157,4,22,510,20,483,0.000000e+00,632,94
3,NP_849578.5,Manes.05G062800.1.p,63.907,471,158,3,22,487,20,483,0.000000e+00,643,94
4,NP_563666.1,Manes.05G062700.1.p,59.792,480,161,4,11,460,10,487,0.000000e+00,604,96
...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,NP_200617.2,Manes.18G145400.1.p,75.941,478,114,1,1,477,1,478,0.000000e+00,762,100
360,NP_201291.1,Manes.12G142500.1.p,72.968,566,142,3,18,576,24,585,0.000000e+00,860,97
361,NP_201319.2,Manes.12G136700.1.p,75.269,372,81,6,1,368,1,365,0.000000e+00,577,99
362,NP_001330820.1,Manes.12G136700.1.p,74.695,328,72,6,1,324,1,321,1.830000e-180,504,97


In [ ]:
# Preparing input protein sequence to 2nd BLASTp 
filename = "input/Mesculenta_305_v6.1.protein.fa"
fasta_record = SeqIO.to_dict(SeqIO.parse(filename, "fasta"))

select_fasta = ''
for gene in first_BLASTp['sseqid'].unique():
    select_fasta = select_fasta + '>' + fasta_record[gene].name + '\n' + fasta_record[gene].seq + '\n'

f = open("input/besthit_ara_cas.fasta", "w")
f.write(str(select_fasta))
f.close()

In [ ]:
# Make cassava protein database for BLASTp
!bin/makeblastdb -in input/GCF_000001735.3_TAIR10_protein.faa \
            -dbtype prot \
            -title AthalianaDB \
            -out input/AthalianaDB
!bin/blastp -db input/AthalianaDB \
        -query input/besthit_ara_cas.fasta \
        -evalue 1e-10 \
        -max_target_seqs 1 \
        -num_threads 2 \
        -outfmt "6 std qcovhsp" \
        -out cas_ara.txt



Building a new DB, current time: 12/03/2020 16:35:18
New DB name:   /home/golf/input/AthalianaDB
New DB title:  AthalianaDB
Sequence type: Protein
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 48350 sequences in 0.816758 seconds.




In [ ]:
second_BLASTp = pd.read_table("cas_ara.txt", 
                             names=["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore", "qcovhsp"], 
                             header=None)

In [ ]:
reciprocal_blast_result = pd.merge(first_BLASTp.filter(["qseqid", "sseqid"]).drop_duplicates(),
         second_BLASTp.filter(["qseqid", "sseqid"]).drop_duplicates(), 
         how='inner', 
         left_on="sseqid", 
         right_on="qseqid",
         suffixes=["_1st_BLASTp", "_2nd_BLASTp"]
        )
reciprocal_blast_result

,qseqid_1st_BLASTp,sseqid_1st_BLASTp,qseqid_2nd_BLASTp,sseqid_2nd_BLASTp
0,NP_171779.1,Manes.01G221800.1.p,Manes.01G221800.1.p,NP_192138.1
1,NP_192138.1,Manes.01G221800.1.p,Manes.01G221800.1.p,NP_192138.1
2,NP_001328883.1,Manes.01G221800.1.p,Manes.01G221800.1.p,NP_192138.1
3,NP_973746.3,Manes.05G062800.1.p,Manes.05G062800.1.p,NP_849578.5
4,NP_001117217.1,Manes.05G062800.1.p,Manes.05G062800.1.p,NP_849578.5
...,...,...,...,...
356,NP_199996.1,Manes.06G141400.1.p,Manes.06G141400.1.p,NP_199996.1
357,NP_200268.3,Manes.15G079300.1.p,Manes.15G079300.1.p,NP_200268.3
358,NP_001330205.1,Manes.15G079300.1.p,Manes.15G079300.1.p,NP_200268.3
359,NP_001330204.1,Manes.15G079300.1.p,Manes.15G079300.1.p,NP_200268.3


In [ ]:
reciprocal_blast_result = reciprocal_blast_result[reciprocal_blast_result["qseqid_1st_BLASTp"] == reciprocal_blast_result["sseqid_2nd_BLASTp"]].filter(["qseqid_1st_BLASTp", "sseqid_1st_BLASTp"])
reciprocal_blast_result.rename(columns={"qseqid_1st_BLASTp": "template_protein_id", 
                                        "sseqid_1st_BLASTp": "cassava_protein_id"}, inplace=True)
reciprocal_blast_result

,template_protein_id,cassava_protein_id
1,NP_192138.1,Manes.01G221800.1.p
5,NP_849578.5,Manes.05G062800.1.p
19,NP_973751.1,Manes.05G073400.1.p
20,NP_171984.2,Manes.15G055400.1.p
22,NP_001322344.1,Manes.05G208000.1.p
...,...,...
353,NP_199959.2,Manes.06G135400.1.p
354,NP_199995.1,Manes.06G141300.1.p
356,NP_199996.1,Manes.06G141400.1.p
357,NP_200268.3,Manes.15G079300.1.p


## 3. Functional assignment

In [ ]:
gene_annotation = reciprocal_blast_result.merge(template_protein[['product_accession', 'ec']], left_on="template_protein_id", right_on="product_accession") \
    .filter(['template_protein_id', 'cassava_protein_id', 'ec'])
gene_annotation.to_csv("cassava_gene_annotation.txt", sep='\t', index=False)
gene_annotation

,template_protein_id,cassava_protein_id,ec
0,NP_192138.1,Manes.01G221800.1.p,ec:3.2.1.4
1,NP_849578.5,Manes.05G062800.1.p,ec:3.2.1.21
2,NP_973751.1,Manes.05G073400.1.p,ec:3.2.1.68
3,NP_171984.2,Manes.15G055400.1.p,ec:2.4.1.14
4,NP_001322344.1,Manes.05G208000.1.p,ec:2.7.7.27
...,...,...,...
89,NP_199959.2,Manes.06G135400.1.p,ec:3.1.3.12
90,NP_199995.1,Manes.06G141300.1.p,ec:5.4.2.2
91,NP_199996.1,Manes.06G141400.1.p,ec:2.7.1.4
92,NP_200268.3,Manes.15G079300.1.p,ec:3.2.1.21
